In [2]:
import pandas as pd
import spotipy
import time
from spotipy.oauth2 import SpotifyClientCredentials

## Spotify's Authorizationn Flow via spotipy

In [3]:
cid = '08ee90c081644cafa2db21e4f124fc53'
c_secret = '8ba7fd8531484cddb405c20ffa78f647'

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=c_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = client_credentials_manager.get_access_token()
spotify = spotipy.Spotify(auth=token)

/Users/brendan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  after removing the cwd from sys.path.


## Scrape Query

Spotify's Web API allows for 50 searches per query. Each query has a max offset offset of 2,000.   
The below scrape query grabs the top 2,000 tracks by year between 1970 - 2020.

In [127]:
df = pd.DataFrame()

for year in range(1970, 2021):
    for i in range(-1, 2000, 50):
        try:
            tracks_data = sp.search(q='year:' + str(year), type = 'track', offset = i, limit =50)
        except:
            pass

        for x in range(len(tracks_data['tracks']['items'])):
            audio_features = sp.audio_features(tracks_data['tracks']['items'][x]['uri'])
            features_df = pd.DataFrame.from_dict(audio_features)
            features_df['track_name'] = tracks_data['tracks']['items'][x]['name']
            features_df['artist_name'] = tracks_data['tracks']['items'][x]['artists'][0]['name']
            features_df['popularity'] = tracks_data['tracks']['items'][x]['popularity']
            features_df['year'] = year
            df = pd.concat([df, features_df], ignore_index=True)

    print(f'2,000 tracks from {year} have been scraped successfully...')

df.to_csv('./Data/1970-2020_tracks', index = False)
    

retrying ...3secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...2secs
retrying ...1secs
2,000 tracks from 1970 have been scraped successfully...
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...2secs
retrying ...2secs
2,000 tracks from 1971 have been scraped successfully...
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
2,000 tracks from 1972 have been scraped successfully...
retrying ...2secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
2,000 tracks from 1973 have been scraped successfully...
retrying ...2secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
2,000 tracks from 1974 have been scraped successfull

/Users/brendan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



2,000 tracks from 2016 have been scraped successfully...
2,000 tracks from 2017 have been scraped successfully...
2,000 tracks from 2018 have been scraped successfully...
2,000 tracks from 2019 have been scraped successfully...
2,000 tracks from 2020 have been scraped successfully...


### Checking if the scraped worked; mostly looking for duplicates...

In [129]:
df.shape

(104550, 23)

In [130]:
df.duplicated().value_counts()

False    102000
True       2550
dtype: int64

In [137]:
df['track_name'].nunique()

80325

In [145]:
df.columns

Index([    'acousticness',     'analysis_url',      'artist_name',
           'danceability',      'duration_ms',           'energy',
                     'id', 'instrumentalness',              'key',
               'liveness',         'loudness',             'mode',
             'popularity',      'speechiness',            'tempo',
         'time_signature',       'track_href',       'track_name',
                   'type',              'uri',          'valence',
                   'year',                  0],
      dtype='object')

In [147]:
df['uri'].nunique()

99448

In [149]:
sum(df['uri'].duplicated())

5101